# Structs Introduction

A struct, or structure, is a custom data type that <span style="color:lightgreen">*lets you package together and name multiple related values*</span> that make up a meaningful group.  
We will also learn <span style="color:lightgreen">*struct associated functions, especially the kind of associated functions called methods*</span>, to specify behavior associated with a struct type

# Defining and Instantiating Structs

- **Structs vs Tuples**: Like tuples, the pieces of a struct can be different types. Unlike with tuples, in a struct you’ll name each piece of data so it’s clear what the values mean. We define a struct and its fields like below

In [2]:
struct User {
    active: bool,  // each piece of data is called 'field'
    username: String,
    email: String,
    sign_in_count: u64,
}


Struct Instance: To use a struct after we’ve defined it, we *create an instance of that struct* by specifying concrete values for each of the fields. 

In [5]:
let user1: User = User {
        active: true,
        username: String::from("someusername123"),
        email: String::from("someone@example.com"),
        sign_in_count: 1,
};

Field Accessing: To get a specific value from a struct, we use dot notation. For example, to access this user’s email address, we use `user1.email`. If the instance is mutable, we can change a value by using the dot notation and assigning into a particular field.

In [10]:
fn main() {
    let mut user1 = User {
        active: true,
        username: String::from("someusername123"),
        email: String::from("someone@example.com"),
        sign_in_count: 1,
    };

    user1.email = String::from("anotheremail@example.com");
    println!("{}", user1.email);
}

main()

anotheremail@example.com


()

Note that the entire instance must be mutable; Rust doesn’t allow us to mark only certain fields as mutable.

## Using the Field Init Shorthand

In [14]:
fn build_user(email: String, username: String) -> User {
    User {
        active: true,
        username,  // same with username: username
        email,  // same with email: email
        sign_in_count: 1,
    }
}

let a = build_user(String::from("anotheremail@example.com"), String::from("user1"));

## Creating Instances from Other Instances with Struct Update Syntax

Look at how we create the `user2` below, where the syntax `..` specifies that the remaining fields not explicitly set should have the same value as the fields in the given instance.

In [16]:
struct User {
    active: bool,
    username: String,
    email: String,
    sign_in_count: u64,
}

fn main() {
    // --snip--

    let user1 = User {
        email: String::from("user1@example.com"),
        username: String::from("someusername123"),
        active: true,
        sign_in_count: 1,
    };

    let user2 = User {
        email: String::from("another@example.com"),
        ..user1  // this must come last
    };
}

> 📙 **Note**  
> <span style="color:orange">*Note that **the struct update syntax uses `=` like an assignment and it moves the data, just as we saw in the “Variables and Data Interacting with Move” section**. In this example, we can no longer use `user1` as a whole after creating `user2` because the `String` in the username field of `user1` was moved into `user2`. If we had given `user2` new `String` values for both `email` and `username`, and thus only used the `active` and `sign_in_count` values from `user1`, then `user1` would still be valid after creating `user2` since `active` and `sign_in_count` are types that implement the `Copy` trait, so they are copied into `user2`. However, `String` does not implement `Copy` and was moved into `user2`* </span>

## Using Tuple Structs Without Named Fields to Create Different Types

<span style="color:lightgreen">*Rust also supports structs that look similar to tuples, called tuple structs. **Tuple structs have the added meaning the struct name provides but don’t have names associated with their fields**; rather, they just have the types of the fields.*</span>

In [18]:
struct Color(i32, i32, i32);
struct Point(i32, i32, i32);

let black = Color(0, 0, 0);
let origin = Point(0, 0, 0);

Note that `black` and `origin` are instances of different types, hence, a function that takes a parameter of type `Color` cannot take a `Point` as an argument, even though both types are made up of three `i32` values.

## Unit-Like Structs Without Any Fields

<span style="color:lightgreen">***Unit-like structs** are structs that don’t have any fields and can useful when you need to implement a trait on some type but don’t have any data that you want to store in the type itself.*</span>

In [ ]:
struct AlwaysEqual;

fn main() {
    // Imagine that later we’ll implement behavior for this type such that every instance 
    // of AlwaysEqual is always equal to every instance of any other type, perhaps to 
    // have a known result for testing purposes
    let subject = AlwaysEqual;
}

## Ownership of Struct Data
Why in 
```rust
struct User {
    active: bool,  // each piece of data is called 'field'
    username: String,
    email: String,
    sign_in_count: u64,
}
```
we define `username` and `email` as `String`, not `&str`?  
<span style="color:skyblue">*Because we want each instance of this struct to own all of its data and for that data to be valid for as long as the entire struct is valid.*</span>

<span style="color:orange">*It’s also possible for structs to store references to data owned by something else, but to do so requires the use of **lifetimes**. **Lifetimes ensure that the data referenced by a struct is valid for as long as the struct is**. Let’s say you try to store a reference in a struct without specifying lifetimes, like the following; this won’t work and produce the error `missing lifetime specifier`:*</span>

In [19]:
struct User {
    active: bool,
    username: &str,
    email: &str,
    sign_in_count: u64,
}

fn main() {
    let user1 = User {
        active: true,
        username: "someusername123",
        email: "someone@example.com",
        sign_in_count: 1,
    };
}

Error: missing lifetime specifier

Error: missing lifetime specifier

Error: mismatched types

Error: mismatched types

# An Example Program Using Struct

In [2]:
#[derive(Debug)]
struct Rectangle {
    width: u32,
    height: u32,
}

fn main() {
    let scale = 2;
    let rect1 = Rectangle {
        height: 50,
        width: dbg!(30 * scale), // Note that dbg! macro takes ownership of an expression
                                 // Also, dbg! returns ownership of the expression’s value
                                 // so width will have the value of 60
    };

    dbg!(&rect1);

    println!("The area is {}", area(&rect1))
}

fn area(rec: &Rectangle) -> u32 {
    rec.width * rec.height
}

main()

The area is 3000


[src/lib.rs:6] 30 * scale = 60
[src/lib.rs:11] &rect1 = Rectangle {
    width: 60,
    height: 50,
}


()

Our `area` function is very specific: it only computes the `area` of rectangles and won’t work with any other type.  
Let’s look at how we tight the `area` function into an `area` method defined on only our `Rectangle` type.

# Method Syntax